<a href="https://colab.research.google.com/github/dmsandipan/RAG-EVAL-Project/blob/main/RAG_Eval_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ragas datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing

In [ ]:
!pip install --upgrade fsspec==2024.10.0
!pip install --upgrade gcsfs==2024.10.0

  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.10.0-py3-none-any.whl (179 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.2.0 requires fsspec[http]<=2024.9.0,>=2023.1.0, but you have fsspec 2024.10.0 which is incompatible.


In [ ]:
from datasets import load_dataset
from ragas import EvaluationDataset, evaluate
from ragas.metrics import (
    LLMContextRecall,
    Faithfulness,
    FactualCorrectness,
    SemanticSimilarity,
)
from ragas.llms import LangchainLLMWrapper


In [ ]:
from datasets import load_dataset

rag_dataset = load_dataset("explodinggradients/ragas-wikiqa")

In [ ]:
from IPython.display import display
display(rag_dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'correct_answer', 'incorrect_answer', 'question_id', 'generated_with_rag', 'context', 'generated_without_rag'],
        num_rows: 232
    })
})

In [ ]:

 rag_dataset = rag_dataset.rename_column("question", "user_input")
 rag_dataset = rag_dataset.rename_column("context", "retrieved_contexts")
 rag_dataset = rag_dataset.rename_column("generated_with_rag", "response")
 rag_dataset = rag_dataset.rename_column("correct_answer", "reference")


#rag_dataset = rag_dataset.map(convert_to_list)
eval_dataset = EvaluationDataset.from_hf_dataset(rag_dataset["train"].select(range(10)))
deep_eval_dataset = rag_dataset["train"].select(range(10))

In [ ]:
from transformers import pipeline
from langchain import HuggingFacePipeline
from ragas import EvaluationDataset, evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
from ragas.llms import LangchainLLMWrapper
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
# generate_text = pipeline(
#     model="google/long-t5-tglobal-base",
#     tokenizer="google/long-t5-tglobal-base",
#     task="text2text-generation",
#     #device_map="auto"
# )
# generate_text.model.to("cuda")
# llm = LangchainLLMWrapper(HuggingFacePipeline(pipeline=generate_text))
llm = LangchainLLMWrapper(ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0))

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall, context_entity_recall, answer_similarity, answer_correctness
from ragas import evaluate


score = evaluate(eval_dataset,metrics=[faithfulness, context_precision, context_recall,answer_relevancy],llm=llm)
score_df = score.to_pandas()
score_df

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,faithfulness,context_precision,context_recall
0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,[African immigration to the United States refe...,\nAfrican Americans were immigrated to the Uni...,"As such, African immigrants are to be distingu...",0.875000,1.0,1.0
1,what are points on a mortgage,"[Discount points, also called mortgage points ...",\nPoints on a mortgage are a form of pre-paid ...,"Points, sometimes also called a ""discount poin...",1.000000,1.0,1.0
2,how does interlibrary loan work,"[Interlibrary loan (abbreviated ILL, and somet...",\nInterlibrary loan works by allowing patrons ...,The user makes a request with their local libr...,1.000000,1.0,1.0
3,WHAT IS A FY QUARTER,[April.\n\n\n=== United States ===\n\n\n==== F...,\nA FY quarter is a three-month period within ...,"A fiscal year (or financial year, or sometimes...",0.666667,1.0,0.0
4,who wrote a rose is a rose is a rose,"[The sentence ""Rose is a rose is a rose is a r...","\nGertrude Stein wrote the sentence ""A rose is...","The sentence ""Rose is a rose is a rose is a ro...",1.000000,1.0,1.0
5,what is a notary for,[A notary public (a.k.a. notary or public nota...,\nA notary is a public officer appointed by a ...,A notary public (or notary or public notary) i...,0.500000,1.0,1.0
6,what bacteria grow on macconkey agar,[MacConkey agar is a selective and differentia...,\nMacConkey agar is designed to selectively is...,MacConkey agar is a culture medium designed to...,1.000000,1.0,1.0
7,who played the lead roles in the movie leaving...,"[a call from Ben, who is on his deathbed. Sera...",\nThe lead roles in the movie Leaving Las Vega...,Nicolas Cage stars as a suicidal alcoholic who...,1.000000,1.0,1.0
8,what is a CMM machine,[A coordinate measuring machine (CMM) is a dev...,\nA coordinate measuring machine (CMM) is a de...,A coordinate measuring machine is a device for...,1.000000,1.0,1.0
9,what percentage of water in in the body,"[In physiology, body water is the water conten...",\nThe percentage of water in the body can vary...,Arthur Guyton 's Textbook of Medical Physiolog...,1.000000,1.0,0.0


In [ ]:
!pip install rouge-score nltk google-cloud-translate sentencepiece transformers evaluate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.8 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=5c428a2eede8d1f7f066c35ab295cec0292671f6192847c5cfb5e9498de0b92e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
#from datasets import load_metric
from nltk.translate.bleu_score import sentence_bleu
import nltk
from rouge_score import rouge_scorer
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import evaluate

nltk.download('punkt')

# Load metrics
bleu_metric = evaluate.load("bleu")
meteor_metric = evaluate.load("meteor")
rouge_metric = evaluate.load("rouge")
bertscore_metric = evaluate.load("bertscore")


# Function to calculate scores for a single example
def calculate_scores(example):
    """
    Calculates BLEU, METEOR, ROUGE, and BERTScore for a single example.

    Args:
    example: A dictionary containing 'reference' and 'response' keys.

    Returns:
    A dictionary containing the calculated scores.
    """
    references = [example["reference"].split()]  # Wrap reference in a list for BLEU
    predictions = example["response"].split()

    # Calculate BLEU score
    bleu_score = sentence_bleu(references, predictions)

    # Calculate METEOR score
    meteor_score = meteor_metric.compute(predictions=[example["response"]], references=[example["reference"]])["meteor"]

    # Calculate ROUGE score
    rouge_scores = rouge_metric.compute(predictions=[example["response"]], references=[example["reference"]])

    # Calculate BERTScore
    bertscore_scores = bertscore_metric.compute(predictions=[example["response"]], references=[example["reference"]], lang="en")

    return {
        "bleu": bleu_score,
        "meteor": meteor_score,
        "rouge1": rouge_scores["rouge1"].mid.fmeasure,
        "rouge2": rouge_scores["rouge2"].mid.fmeasure,
        "rougeL": rouge_scores["rougeL"].mid.fmeasure,
        "bertscore": bertscore_scores["f1"][0]  # Assuming you want the F1 score
    }


# Apply the function to your dataset
updated_dataset = rag_dataset["train"].select(range(10)).map(calculate_scores)

# Print the results
print(updated_dataset)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AttributeError: 'numpy.float64' object has no attribute 'mid'

In [ ]:
#from datasets import load_metric
from nltk.translate.bleu_score import sentence_bleu
import nltk
from rouge_score import rouge_scorer
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import evaluate

nltk.download('punkt')

# Load metrics
bleu_metric = evaluate.load("bleu")
meteor_metric = evaluate.load("meteor")
rouge_metric = evaluate.load("rouge")
bertscore_metric = evaluate.load("bertscore")


# Function to calculate scores for a single example
def calculate_scores(example):
    """
    Calculates BLEU, METEOR, ROUGE, and BERTScore for a single example.

    Args:
    example: A dictionary containing 'reference' and 'response' keys.

    Returns:
    A dictionary containing the calculated scores.
    """
    references = [example["reference"].split()]  # Wrap reference in a list for BLEU
    predictions = example["response"].split()

    # Calculate BLEU score
    bleu_score = sentence_bleu(references, predictions)

    # Calculate METEOR score
    meteor_score = meteor_metric.compute(predictions=[example["response"]], references=[example["reference"]])["meteor"]

    # Calculate ROUGE score
    rouge_scores = rouge_metric.compute(predictions=[example["response"]], references=[example["reference"]])

    # Calculate BERTScore
    bertscore_scores = bertscore_metric.compute(predictions=[example["response"]], references=[example["reference"]], lang="en")

    return {
        "bleu": bleu_score,
        "meteor": meteor_score,
        "rouge1": rouge_scores["rouge1"], # Access the fmeasure directly.
        "rouge2": rouge_scores["rouge2"], # Access the fmeasure directly.
        "rougeL": rouge_scores["rougeL"], # Access the fmeasure directly.
        "bertscore": bertscore_scores["f1"][0]  # Assuming you want the F1 score
    }


# Apply the function to your dataset
updated_dataset = rag_dataset["train"].select(range(10)).map(calculate_scores)

# Print the results
print(updated_dataset)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-package

Dataset({
    features: ['user_input', 'correct_answer', 'reference', 'question_id', 'response', 'retrieved_contexts', 'generated_without_rag', 'bleu', 'meteor', 'rouge1', 'rouge2', 'rougeL', 'bertscore'],
    num_rows: 10
})


In [ ]:
updated_dataset.to_pandas().head()

,user_input,correct_answer,reference,question_id,response,retrieved_contexts,generated_without_rag,bleu,meteor,rouge1,rouge2,rougeL,bertscore
0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,"As such, African immigrants are to be distingu...",From the Immigration and Nationality Act of 19...,Q0,\nAfrican Americans were immigrated to the Uni...,[African immigration to the United States refe...,African Americans were immigrated to the US in...,1.021080e-01,0.417134,0.312925,0.193103,0.217687,0.860487
1,what are points on a mortgage,"Points, sometimes also called a ""discount poin...",Discount points may be different from originat...,Q1012,\nPoints on a mortgage are a form of pre-paid ...,"[Discount points, also called mortgage points ...",A mortgage point is a fee equal to 1% of the l...,6.868423e-232,0.089686,0.034483,0.000000,0.034483,0.798285
2,how does interlibrary loan work,The user makes a request with their local libr...,Although books and journal articles are the mo...,Q102,\nInterlibrary loan works by allowing patrons ...,"[Interlibrary loan (abbreviated ILL, and somet...",Interlibrary loan is a service that allows lib...,9.235414e-232,0.189125,0.105263,0.000000,0.060150,0.827930
3,WHAT IS A FY QUARTER,"A fiscal year (or financial year, or sometimes...",Fiscal years vary between businesses and count...,Q1027,\nA FY quarter is a three-month period within ...,[April.\n\n\n=== United States ===\n\n\n==== F...,A FY Quarter is a three-month period in the fi...,7.114448e-232,0.258709,0.078431,0.000000,0.078431,0.803739
4,who wrote a rose is a rose is a rose,"The sentence ""Rose is a rose is a rose is a ro...",I know that in daily life we don't go around s...,Q1032,"\nGertrude Stein wrote the sentence ""A rose is...","[The sentence ""Rose is a rose is a rose is a r...","Gertrude Stein wrote ""A Rose is a Rose is a Ro...",6.867036e-156,0.116325,0.241379,0.107143,0.206897,0.835694


In [ ]:
updated_dataset.to_pandas().to_csv('updated_dataset.csv', index=False)

In [ ]:
from google.colab import files

In [ ]:
from google.colab import files

# Convert the Hugging Face Dataset to a Pandas DataFrame
df = updated_dataset.to_pandas()

# Save the DataFrame to a temporary CSV file in Colab
df.to_csv('updated_dataset.csv', index=False)

# Trigger the download in your browser
files.download('updated_dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.0 MB/s eta 0:00:00


In [ ]:
!pip install datasets --upgrade  # Upgrade datasets to the latest version
!pip install evaluate

In [ ]:
from datasets import load_dataset
from deepeval import evaluate
from deepeval.metrics import ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase
from ragas import EvaluationDataset


deep_eval_dataset = rag_dataset["train"].select(range(10))

# Create test cases for deepeval
test_cases = []
for example in deep_eval_dataset:  # Iterate directly through the Dataset
    test_cases.append(
        LLMTestCase(
            input=example["user_input"],
            actual_output=example["response"],
            retrieval_context=example["retrieved_contexts"]
        )
    )
#print(test_cases)
# Initialize the metric
metric = ContextualRelevancyMetric(
    threshold=0.7,
    model="gpt-3.5-turbo",  # Or use a different model if available
    include_reason=True
)

# Evaluate and print results
results = evaluate(test_cases, [metric])


✨ You're running DeepEval's latest Contextual Relevancy Metric! (using gpt-3.5-turbo, strict=False, 
async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 10 test case(s) in parallel: |██████████|100% (10/10) [Time Taken: 00:19,  1.91s/test case]



Metrics Summary

  - ❌ Contextual Relevancy (score: 0.625, threshold: 0.7, strict: False, evaluation model: gpt-3.5-turbo, reason: The score is 0.62 because while the irrelevant statements provide specific details not directly related to the definition of a CMM machine, the relevant statements offer accurate information about CMMs and their measurement technology., error: None)

For test case:

  - input: what is a CMM machine
  - actual output: 
A coordinate measuring machine (CMM) is a device that measures the geometry of physical objects by sensing discrete points on the surface of the object with a probe. It typically specifies a probe's position in terms of its displacement from a reference position in a three-dimensional Cartesian coordinate system (i.e., with XYZ axes). It is commonly used in manufacturing and assembly processes to test a part or assembly against the design intent, and can be used to measure the distance between features, construct geometric features such as c

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

In [ ]:
!pip install deepeval

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 576.5/576.5 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 3.4 MB/s eta 0:00:00
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=a1ff61c6923e820a28e53095704bb49c400b32f53ec3b515bac38a0c6fce7f84
  

With Wrong Reference


In [ ]:
rag_dataset = rag_dataset.rename_column("question", "user_input")
rag_dataset = rag_dataset.rename_column("context", "retrieved_contexts")
rag_dataset = rag_dataset.rename_column("generated_with_rag", "response")
rag_dataset = rag_dataset.rename_column("incorrect_answer", "reference")


#rag_dataset = rag_dataset.map(convert_to_list)
eval_dataset = EvaluationDataset.from_hf_dataset(rag_dataset["train"].select(range(10)))
deep_eval_dataset = rag_dataset["train"].select(range(10))

In [ ]:
from transformers import pipeline
from langchain import HuggingFacePipeline
from ragas import EvaluationDataset, evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
from ragas.llms import LangchainLLMWrapper
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
# generate_text = pipeline(
#     model="google/long-t5-tglobal-base",
#     tokenizer="google/long-t5-tglobal-base",
#     task="text2text-generation",
#     #device_map="auto"
# )
# generate_text.model.to("cuda")
# llm = LangchainLLMWrapper(HuggingFacePipeline(pipeline=generate_text))
llm = LangchainLLMWrapper(ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0))

context precision is between context and answer

context recall is between context and ground truth

context relevancy is between context and query

answer relevancy is between answer and query

faithfulness is between answer and context


In [ ]:
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall, context_entity_recall, answer_similarity, answer_correctness
from ragas import evaluate


score = evaluate(eval_dataset,metrics=[faithfulness, context_precision, context_recall,answer_relevancy],llm=llm)
score_df = score.to_pandas()
score_df

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,faithfulness,context_precision,context_recall,answer_relevancy
0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,[African immigration to the United States refe...,\nAfrican Americans were immigrated to the Uni...,From the Immigration and Nationality Act of 19...,0.875000,1.0,0.0,0.913350
1,what are points on a mortgage,"[Discount points, also called mortgage points ...",\nPoints on a mortgage are a form of pre-paid ...,Discount points may be different from originat...,1.000000,1.0,1.0,0.930753
2,how does interlibrary loan work,"[Interlibrary loan (abbreviated ILL, and somet...",\nInterlibrary loan works by allowing patrons ...,Although books and journal articles are the mo...,1.000000,1.0,0.0,0.975837
3,WHAT IS A FY QUARTER,[April.\n\n\n=== United States ===\n\n\n==== F...,\nA FY quarter is a three-month period within ...,Fiscal years vary between businesses and count...,0.666667,1.0,1.0,0.868936
4,who wrote a rose is a rose is a rose,"[The sentence ""Rose is a rose is a rose is a r...","\nGertrude Stein wrote the sentence ""A rose is...",I know that in daily life we don't go around s...,1.000000,1.0,1.0,0.802378
5,what is a notary for,[A notary public (a.k.a. notary or public nota...,\nA notary is a public officer appointed by a ...,"Therefore, at common law, notarial service is ...",0.500000,1.0,1.0,0.879704
6,what bacteria grow on macconkey agar,[MacConkey agar is a selective and differentia...,\nMacConkey agar is designed to selectively is...,A MacConkey agar plate with an active bacteria...,0.933333,1.0,1.0,0.879251
7,who played the lead roles in the movie leaving...,"[a call from Ben, who is on his deathbed. Sera...",\nThe lead roles in the movie Leaving Las Vega...,Leaving Las Vegas was filmed in super 16mm ins...,1.000000,0.5,0.5,0.922367
8,what is a CMM machine,[A coordinate measuring machine (CMM) is a dev...,\nA coordinate measuring machine (CMM) is a de...,"Probes may be mechanical, optical, laser, or w...",1.000000,1.0,1.0,0.899232
9,what percentage of water in in the body,"[In physiology, body water is the water conten...",\nThe percentage of water in the body can vary...,"Per Guyton, in a body containing 40 litres of ...",1.000000,1.0,0.0,0.942190


In [ ]:
from google.colab import data_table
data_table.disable_dataframe_formatter()
score_df

NameError: name 'score_df' is not defined